<a href="https://colab.research.google.com/github/madarasw/Advanced_deep_learning_module/blob/main/Class_Project/ViTs_and_FFT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Test 2 - with FFT


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!pip install kaggle

In [4]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle (1).json


{'kaggle (1).json': b'{"username":"madarawimalarathna","key":"998571907b3220ce6ac5e9cd10df1deb"}'}

In [5]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [6]:
!ls -l ~/.kaggle

total 4
-rw------- 1 root root 74 Nov  6 11:52 kaggle.json


In [6]:
!kaggle datasets download -d ciplab/real-and-fake-face-detection

Dataset URL: https://www.kaggle.com/datasets/ciplab/real-and-fake-face-detection
License(s): CC-BY-NC-SA-4.0
 89% 384M/431M [00:04<00:00, 58.0MB/s]
100% 431M/431M [00:04<00:00, 109MB/s] 


In [7]:
!unzip real-and-fake-face-detection.zip -d data/

Archive:  real-and-fake-face-detection.zip
  inflating: data/real_and_fake_face/training_fake/easy_100_1111.jpg  
  inflating: data/real_and_fake_face/training_fake/easy_101_0010.jpg  
  inflating: data/real_and_fake_face/training_fake/easy_102_0101.jpg  
  inflating: data/real_and_fake_face/training_fake/easy_103_1111.jpg  
  inflating: data/real_and_fake_face/training_fake/easy_104_1000.jpg  
  inflating: data/real_and_fake_face/training_fake/easy_105_1100.jpg  
  inflating: data/real_and_fake_face/training_fake/easy_106_0011.jpg  
  inflating: data/real_and_fake_face/training_fake/easy_107_1011.jpg  
  inflating: data/real_and_fake_face/training_fake/easy_108_1111.jpg  
  inflating: data/real_and_fake_face/training_fake/easy_109_1111.jpg  
  inflating: data/real_and_fake_face/training_fake/easy_10_0001.jpg  
  inflating: data/real_and_fake_face/training_fake/easy_110_0001.jpg  
  inflating: data/real_and_fake_face/training_fake/easy_111_1101.jpg  
  inflating: data/real_and_fake_fac

In [1]:
!pip uninstall -y transformers accelerate
!rm -rf /usr/local/lib/python3.12/dist-packages/transformers
!rm -rf /usr/local/lib/python3.12/dist-packages/transformers-*
!rm -rf /usr/local/lib/python3.12/dist-packages/accelerate*

Found existing installation: transformers 4.45.1
Uninstalling transformers-4.45.1:
  Successfully uninstalled transformers-4.45.1
Found existing installation: accelerate 1.2.0
Uninstalling accelerate-1.2.0:
  Successfully uninstalled accelerate-1.2.0


In [3]:
!pip install transformers==4.45.1 accelerate==1.2.0 datasets==2.19.1

  Using cached transformers-4.45.1-py3-none-any.whl.metadata (44 kB)
  Using cached accelerate-1.2.0-py3-none-any.whl.metadata (19 kB)
Using cached transformers-4.45.1-py3-none-any.whl (9.9 MB)
Using cached accelerate-1.2.0-py3-none-any.whl (336 kB)


In [2]:
!pip install "numpy<2.0"

In [4]:
import transformers
print("Transformers loaded from:", transformers.__file__)

Transformers loaded from: /usr/local/lib/python3.12/dist-packages/transformers/__init__.py


In [5]:
from transformers import TrainingArguments
import inspect

print(inspect.getfile(TrainingArguments))

/usr/local/lib/python3.12/dist-packages/transformers/training_args.py


# Script

In [6]:
"""
finetune_vit_with_fft_fusion.py

Single-file pipeline:
 - Load image folder dataset (ImageFolder layout or GenImage subset)
 - Optional: create train/val/test split (70/15/15)
 - Preprocess images with ViTImageProcessor (resizing, normalization)
 - Optionally augment training images
 - Fine-tune a pretrained ViT for binary classification
 - Optional: fuse ViT features with FFT + small CNN front-end
 - Evaluate with accuracy / precision / recall / f1 / auc
"""

import os
import random
from pathlib import Path
from typing import Dict, Tuple, Optional

import numpy as np
from PIL import Image
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F

from transformers import (
    ViTForImageClassification,
    ViTModel,
    ViTImageProcessor,
    TrainingArguments,
    Trainer,
)
from datasets import load_dataset, DatasetDict, Image, load_from_disk, Dataset
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score

In [7]:
# ----------------------- CONFIG -----------------------
# Edit these


RAW_DATA_ROOT = "/content/data/real_and_fake_face_detection/real_and_fake_face"
PROCESSED_DATA = "/content/data_split"   # ✅ where ImageFolder structure will go
OUTPUT_DIR = "/content/vit_output"


DATA_ROOT = "/content/data/real_and_fake_face_detection/" # path to ImageFolder-style dataset root (train/class/... or single folder to split)
MODEL_NAME = "google/vit-base-patch16-224"  # or "facebook/deit-base-distilled-patch16-224"

USE_FUSION = True            # fuse ViT with FFT + small CNN front-end
IMAGE_SIZE = 224
BATCH_SIZE = 16
EPOCHS = 6
LR = 5e-5
WEIGHT_DECAY = 0.01
SEED = 42
TRAIN_VAL_TEST_SPLIT = (0.7, 0.15, 0.15)   # if dataset contains only a single folder; otherwise expects train/val/test under DATA_ROOT
# ------------------------------------------------------


In [13]:
# ---------------- utilities ----------------
def set_seed(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(SEED)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# ---------------- Prepare dataset ----------------
# The script expects ImageFolder layout:
# DATA_ROOT/
#   train/
#     real/
#     fake/
#   val/
#     real/
#     fake/
#   test/
#     real/
#     fake/
#
# If DATA_ROOT contains `train/` and `val/` use them. If it contains only class folders, we'll split.
def prepare_datasets(data_root: str, split: Tuple[float, float, float]=(0.7,0.15,0.15)) -> Tuple[DatasetDict, Dict]:
    """
    Returns a HuggingFace DatasetDict with 'train','validation','test' and a label2id mapping.
    If folders train/val/test exist under data_root they'll be used. Otherwise will split from data_root.
    """
    data_root = Path(data_root)
    # Look for train/val/test
    if (data_root / "train").exists():
        # assume structure exists already
        print("Using existing train/val/test structure under", data_root)
        ds_train = load_dataset("imagefolder", data_dir=str(data_root / "train"))["train"]
        ds_val = load_dataset("imagefolder", data_dir=str(data_root / "val"))["train"] if (data_root / "val").exists() else None
        ds_test = load_dataset("imagefolder", data_dir=str(data_root / "test"))["train"] if (data_root / "test").exists() else None

        if ds_val is None or ds_test is None:
            # if test missing, split train
            print("No complete val/test found. Splitting train into train/validation/test.")
            ds_all = ds_train
            # stratified-ish split by label
            train_frac, val_frac, test_frac = split
            ds_all = ds_all.train_test_split(test_size=(1.0 - train_frac), seed=SEED, stratify_by_column="label")
            # ds_all['train'] and ds_all['test'] (which contains val+test)
            # take ds_all['test'] and split into val/test proportionally
            remaining = ds_all['test']
            val_ratio = val_frac / (val_frac + test_frac)
            rem_split = remaining.train_test_split(test_size=(1.0 - val_ratio), seed=SEED, stratify_by_column="label")
            ds = DatasetDict({
                "train": ds_all['train'],
                "validation": rem_split['train'],
                "test": rem_split['test']
            })
        else:
            ds = DatasetDict({"train": ds_train, "validation": ds_val, "test": ds_test})

    else:
        # assume top-level has class folders -> use imagefolder on data_root and split
        print("No train/val/test directories found. Loading from top-level and splitting.")
        ds_all = load_dataset("imagefolder", data_dir=str(data_root))["train"]
        train_frac, val_frac, test_frac = split
        # First split off train
        ds_tmp = ds_all.train_test_split(test_size=(1.0 - train_frac), seed=SEED, stratify_by_column="label")
        remaining = ds_tmp['test']
        # split remaining into val/test
        val_ratio = val_frac / (val_frac + test_frac)
        rem_split = remaining.train_test_split(test_size=(1.0 - val_ratio), seed=SEED, stratify_by_column="label")
        ds = DatasetDict({
            "train": ds_tmp['train'],
            "validation": rem_split['train'],
            "test": rem_split['test']
        })

    label_list = ds["train"].features["label"].names
    label2id = {name: i for i, name in enumerate(label_list)}
    print("Labels:", label_list)
    return ds, {"label2id": label2id, "id2label": {v:k for k,v in label2id.items()}}

# call
dataset, label_maps = prepare_datasets(DATA_ROOT, split=TRAIN_VAL_TEST_SPLIT)

# ---------------- Processor / augmentation ----------------
image_processor = ViTImageProcessor.from_pretrained(MODEL_NAME)

# We will define preprocess functions that use torchvision transforms for augmentation (train), then apply image_processor to get pixel_values
import torchvision.transforms as T

train_augment = T.Compose([
    T.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    T.RandomHorizontalFlip(),
    T.RandomRotation(10),
    T.ColorJitter(0.1,0.1,0.1),
])

val_transform = T.Compose([
    T.Resize((IMAGE_SIZE, IMAGE_SIZE)),
])

from PIL import Image
print(Image.Image)

def pil_to_np(img: Image.Image):
    return np.array(img.convert("RGB"))

def preprocess_examples(examples, stage="train"):
    imgs = []
    for p in examples["image"]:
        if isinstance(p, dict) and "path" in p:
            # datasets returns an 'Image' feature, which may be a dict with path if loaded lazily
            img = Image.open(p["path"]).convert("RGB")
        elif isinstance(p, Image.Image):
            img = p
        else:
            # fallback
            img = Image.open(p).convert("RGB")
        if stage == "train":
            img = train_augment(img)
        else:
            img = val_transform(img)
        imgs.append(np.array(img))
    # use ViTImageProcessor to produce pixel_values (float32, normalized)
    processed = image_processor(images=imgs, return_tensors="np")
    examples["pixel_values"] = processed["pixel_values"]
    examples["label"] = examples["label"]
    return examples

# map with batched=True; ensure remove original image column to save space later
# dataset["train"] = dataset["train"].with_transform(lambda example: {"image": example["image"], "label": example["label"]})
# dataset["validation"] = dataset["validation"].with_transform(lambda example: {"image": example["image"], "label": example["label"]})
# dataset["test"] = dataset["test"].with_transform(lambda example: {"image": example["image"], "label": example["label"]})



# We'll use map with batched processing to build pixel_values (numpy arrays)
print("Preprocessing training set (this may take a while)...")
# Keep the label column — remove only image column
remove_cols = [c for c in dataset["train"].column_names if c != "label"]

dataset["train"] = dataset["train"].map(
    lambda ex: preprocess_examples(ex, stage="train"),
    batched=True,
    remove_columns=remove_cols
)
dataset["validation"] = dataset["validation"].map(
    lambda ex: preprocess_examples(ex, stage="val"),
    batched=True,
    remove_columns=remove_cols
)
dataset["test"] = dataset["test"].map(
    lambda ex: preprocess_examples(ex, stage="val"),
    batched=True,
    remove_columns=remove_cols
)

# datasets now have 'pixel_values' as numpy arrays and 'label'
# But Trainer expects torch.Tensor in collate_fn; we'll write a data collator below.

# ---------------- Custom Model: ViT or ViT+CNN+FFT fusion ----------------
# We'll create a model class that returns logits and (optionally) accepts pixel_values
class ViTFFTClassifier(nn.Module):
    def __init__(self, model_name: str, num_labels: int=2, use_fusion: bool=True):
        super().__init__()
        self.use_fusion = use_fusion
        # ViT encoder (without head)
        self.vit = ViTModel.from_pretrained(model_name)
        vit_hidden = self.vit.config.hidden_size  # usually 768
        # classification head if vanilla
        if not use_fusion:
            self.classifier = nn.Linear(vit_hidden, num_labels)
        else:
            # small CNN front-end -> produce feature map
            self.cnn_front = nn.Sequential(
                nn.Conv2d(3, 32, kernel_size=3, padding=1),
                nn.ReLU(),
                nn.Conv2d(32, 32, kernel_size=3, padding=1),
                nn.ReLU(),
                nn.AdaptiveAvgPool2d((14,14))
            )
            cnn_feat_dim = 32 * 14 * 14
            # FFT branch: project magnitude spectrum
            # we flatten grayscale magnitude and project to 512 dims
            self.fft_proj = nn.Linear(IMAGE_SIZE * IMAGE_SIZE, 512)
            fusion_dim = vit_hidden + cnn_feat_dim + 512
            self.fusion_proj = nn.Linear(fusion_dim, vit_hidden)
            self.classifier = nn.Linear(vit_hidden, num_labels)

    def fft_features(self, pixel_values: torch.Tensor):
        # pixel_values: [B, C, H, W] (float, normalized). We'll convert to grayscale approx and compute fft
        # Undo normalization approximate? We can compute FFT on normalized images — artifact patterns persist.
        gray = pixel_values.mean(dim=1)  # [B, H, W]
        fft = torch.fft.fft2(gray)       # complex
        mag = torch.abs(fft)
        # we may want log scaling
        mag = torch.log1p(mag)
        B, H, W = mag.shape
        mag_flat = mag.view(B, -1)       # [B, H*W]
        return mag_flat

    def forward(self, pixel_values: torch.Tensor, labels: Optional[torch.Tensor]=None):
        # pixel_values is expected as float tensor [B, C, H, W]
        outputs = self.vit(pixel_values).last_hidden_state  # [B, seq_len, hidden]
        cls = outputs[:, 0, :]   # CLS token
        if not self.use_fusion:
            logits = self.classifier(cls)
        else:
            # CNN front-end expects unnormalized images; but works with normalized too
            cnn_feats = self.cnn_front(pixel_values)  # [B, Cf, 14, 14]
            cnn_flat = cnn_feats.view(cnn_feats.size(0), -1)
            fft_flat = self.fft_features(pixel_values)  # [B, H*W]
            fft_proj = self.fft_proj(fft_flat)  # [B, 512]
            fused = torch.cat([cls, cnn_flat, fft_proj], dim=1)
            fused_proj = self.fusion_proj(fused)  # project back to vit_hidden
            combined = cls + fused_proj
            logits = self.classifier(combined)
        loss = None
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits, labels)
            return {"loss": loss, "logits": logits}
        return {"logits": logits}

# ---------------- Data collator ----------------
# dataset features: 'pixel_values' is numpy array shape (B, C, H, W), 'label' int
def collate_fn(batch):
    # batch is list of examples (each example: {'pixel_values': np.array(...), 'label': int})
    pixel_values = np.stack([ex["pixel_values"] for ex in batch])  # (B, C, H, W)
    labels = np.array([ex["label"] for ex in batch])
    pixel_values = torch.tensor(pixel_values).float()
    labels = torch.tensor(labels).long()
    return {"pixel_values": pixel_values, "labels": labels}

# ---------------- Compute metrics ----------------
def compute_metrics(pred):
    logits = pred.predictions
    labels = pred.label_ids
    probs = torch.softmax(torch.tensor(logits), dim=1).numpy()
    preds = np.argmax(logits, axis=1)
    acc = accuracy_score(labels, preds)
    prec, rec, f1, _ = precision_recall_fscore_support(labels, preds, average="binary", zero_division=0)
    try:
        auc = roc_auc_score(labels, probs[:,1])
    except Exception:
        auc = 0.0
    return {"accuracy": acc, "precision": prec, "recall": rec, "f1": f1, "roc_auc": auc}

# ---------------- Build model and Trainer ----------------
num_labels = len(set(dataset["train"]["label"]))
print("Num labels:", num_labels)

model = ViTFFTClassifier(MODEL_NAME, num_labels=num_labels, use_fusion=USE_FUSION)
model.to(device)

# Wrap model in a simple HF-compatible wrapper for Trainer
# Trainer expects a model object with forward(**inputs) returning loss and logits (or just logits)
# We'll make a small adapter that calls our model and returns loss when labels present.
class HFModelWrapper(nn.Module):
    def __init__(self, base_model: ViTFFTClassifier):
        super().__init__()
        self.base = base_model
    def forward(self, pixel_values, labels=None):
        pixel_values = pixel_values.to(device)
        if labels is not None:
            labels = labels.to(device)
        out = self.base(pixel_values, labels)
        if labels is not None:
            return torch.tensor(out["loss"].cpu().detach().numpy()).to(device) if isinstance(out["loss"], torch.Tensor) else out["loss"]
        # Trainer expects either a ModelOutput or a tuple (loss, logits). We'll return logits only here.
        return out["logits"]

# But Trainer can work with our base model directly if we build a custom compute_loss; to avoid complexity, we'll use a custom Trainer subclass.
from transformers import Trainer as HfTrainer, TrainingArguments

class CustomTrainer(HfTrainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        pixel_values = inputs["pixel_values"].to(device)
        labels = inputs["labels"].to(device)
        out = model(pixel_values, labels)
        loss = out["loss"]
        if return_outputs:
            return loss, out
        return loss

# TrainingArguments
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=EPOCHS,
    learning_rate=LR,
    weight_decay=0.01,

    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="steps",

    logging_steps=50,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    report_to="none"
)

# Create Trainer
trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    tokenizer=None,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
)

# ---------------- Train ----------------
trainer.train()

# ---------------- Eval on test ----------------
metrics = trainer.evaluate(dataset["test"])
print("Test metrics:", metrics)

# Save final model
trainer.save_model(os.path.join(OUTPUT_DIR, "final_model"))


Using device: cuda
No train/val/test directories found. Loading from top-level and splitting.


Resolving data files:   0%|          | 0/2041 [00:00<?, ?it/s]

Labels: ['training_fake', 'training_real']
<class 'PIL.Image.Image'>
Preprocessing training set (this may take a while)...


Map:   0%|          | 0/1428 [00:00<?, ? examples/s]

Map:   0%|          | 0/306 [00:00<?, ? examples/s]

Map:   0%|          | 0/307 [00:00<?, ? examples/s]

Num labels: 2


Some weights of ViTModel were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized: ['vit.pooler.dense.bias', 'vit.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.12/dist-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1,Roc Auc
1,1.578200,0.524134,0.761438,0.732984,0.864198,0.793201,0.832090
2,0.271400,0.624158,0.715686,0.686567,0.851852,0.760331,0.820516
3,0.096100,0.864210,0.771242,0.865079,0.672840,0.756944,0.871271
4,0.010900,0.876705,0.830065,0.808989,0.888889,0.847059,0.881216
5,0.001800,0.953355,0.810458,0.785714,0.882716,0.831395,0.874057
6,0.000200,0.934849,0.816993,0.794444,0.882716,0.836257,0.878194


Test metrics: {'eval_loss': 0.8939898014068604, 'eval_accuracy': 0.8045602605863192, 'eval_precision': 0.7754010695187166, 'eval_recall': 0.8895705521472392, 'eval_f1': 0.8285714285714286, 'eval_roc_auc': 0.8853953646898431, 'eval_runtime': 33.9888, 'eval_samples_per_second': 9.032, 'eval_steps_per_second': 0.588, 'epoch': 6.0}
